Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

## Background
ADME@NCATS is a resource developed by NCATS to host in silico prediction models for various ADME (Absorption, Distribution, Metabolism and Excretion) properties. The resource serves as an important tool for the drug discovery community with potential uses in compound optimization and prioritization. The models were retrospectively validated on a subset of marketed drugs which resulted in very good accuracies.

Data that were used for developing the models are made publicly accessible by depositing them into PubChem database. In some instances, when complete data cannot be made public, a subset of the data are deposited into PubChem. Links to the PubChem assays can be found in the individual model pages. The users are highly encouraged to use these data for development and validation of QSAR models.

## Assay Information
Hepatic metabolic stability is a key pharmacokinetic parameter in drug discovery. Metabolic stability is usually assessed in microsomal fractions and only the best compounds progress in the drug discovery process. A high-throughput single time point substrate depletion method in rat liver microsomes (RLM) is employed at the National Center for Advancing Translational Sciences (NCATS) as a Tier 1 assay. Between 2012 and 2020, RLM stability (in vitro half-life) data was generated for ~24,000 compounds from more than 250 NCATS projects that cover a wide range of pharmacological targets and cellular pathways. Data for ~2500 compounds along with the global prediction models are publicly available.

![image.png](https://storage.googleapis.com/polaris-public/icons/icon_adme_ncats.jpg)

Image is from [this paper](https://link.springer.com/article/10.1007/s00216-016-9929-6). Biologically active compounds can be transformed or destroyed by the action of enzymes in the liver. Microsomes are small membrane bubbles (vesicles) that come from a fragmented cell membrane, and can be used as a proxy for how well a drug survives a trip through the liver.

## Description of readout:
- **PUBCHEM_ACTIVITY_OUTCOME**: Corresponds to the phenotype observed. For all compounds with phenotype "stable", PUBCHEM_ACTIVITY_OUTCOME is "active" (class = 1). For all compounds with phenotype "unstable", PUBCHEM_ACTIVITY_OUTCOME is "inactive" (class = 0).
- **PHENOTYPE**: Based on the half-life observed. Compound is "stable" if half-life (t1/2) is more than 30 minutes (class = 1); "unstable" if t1/2 is less than 30 minutes (class = 0).
- **HALF-LIFE**: Rat liver microsome stability (t1/2), in minutes. >30 minutes has been converted to 31.

**Optimization objective**: A drug with high hepatic metabolic stability is often desirable as it can result in a more sustained therapeutic effect with lower dosing frequencies. However, excessive metabolic stability can lead to drug accumulation and potential toxicity. There may not be a universally applicable optimal range for hepatic metabolic stability, the goal is to achieve a balance that maximizes therapeutic benefit while minimizing adverse effects and maintaining an acceptable safety profile.

In the context of this dataset, `stable` compounds are preferable.

## Data resource

**Reference**: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7693334/ 

**Raw data**: https://pubchem.ncbi.nlm.nih.gov/bioassay/1508591

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

root = pathlib.Path("__file__").absolute().parents[3]
# set to recipe root directory
os.chdir(root)
sys.path.insert(0, str(root))

In [6]:
org = "polaris"
data_name = "ncats_adme/RLM"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

All datasets were downloaded directly from Pubchem on 2024-03-21 by following the PubChem Bioassay links on https://opendata.ncats.nih.gov/adme/data.

In [7]:
# Load the data
source_data_path = f"{gcp_root}/data/raw/AID_1508591_raw.parquet"
data = pd.read_parquet(source_data_path)

Rows 0 and 1 are metadata; we will keep them separate.

In [8]:
meta_start = 0  # Start row index
meta_end = 2  # End row index + 1
data_meta = data.iloc[meta_start:(meta_end), :].copy()  # Save the metadata rows
data = data.drop(labels=list(range(meta_start, meta_end)), axis=0).reset_index(
    drop=True
)  # Drop those rows from the main dataframe
data_meta

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Phenotype,Half-life (minutes),Analysis Comment,Compound QC
0,RESULT_TYPE,NaN,NaN,None,None,NaN,NaN,NaN,STRING,STRING,STRING,STRING
1,RESULT_DESCR,NaN,NaN,None,None,NaN,NaN,NaN,Indicates type of activity observed: Unstable ...,Rat liver microsome stability (T1/2),Annotation/notes on a particular compound's da...,Source of compound QC


Drop the metadata rows and keep only the smiles, ID and outcome rows. We'll also turn all columns to uppercase for consistency and rename columns.

In [9]:
# Keep only the SMILES, ID and outcome rows
columns_to_keep = [
    "PUBCHEM_SID",
    "PUBCHEM_EXT_DATASOURCE_SMILES",
    "PUBCHEM_ACTIVITY_OUTCOME",
    "Phenotype",
    "Half-life (minutes)",
]
data = data[columns_to_keep]
data.rename(columns={"PUBCHEM_EXT_DATASOURCE_SMILES": "SMILES"}, inplace=True)
# Rename all columns to uppercase
for col in data.columns:
    data.rename(columns={col: col.upper()}, inplace=True)
# Rename half-life (minutes) to half-life (we will specify minutes in the metadata)
data.rename(columns={"HALF-LIFE (MINUTES)": "HALF-LIFE"}, inplace=True)

### Map values to digits and replace cutoffs
Half-life is given in minutes but anything over 30 minutes was treated as '>30'. To help with machine readability, we will convert '>30' to 31.0. We will also convert Active/Inactive and Stable/Unstable to binary (1/0 respectively).

In [10]:
# Map active/inactive and stable/unstable to 1 and 0
data["PUBCHEM_ACTIVITY_OUTCOME"] = data["PUBCHEM_ACTIVITY_OUTCOME"].map(
    {"Active": 1.0, "Inactive": 0.0}
)
data["PHENOTYPE"] = data["PHENOTYPE"].map({"stable": 1.0, "unstable": 0.0})
# Replace >30 with 31 so it can be operated on
data["HALF-LIFE"].replace(">30", "31.0", inplace=True)
# Convert types after replacement
data["HALF-LIFE"] = data["HALF-LIFE"].astype(float)

In [11]:
data.describe(include='all')

,PUBCHEM_SID,SMILES,PUBCHEM_ACTIVITY_OUTCOME,PHENOTYPE,HALF-LIFE
count,2.531000e+03,2531,2531.000000,2531.000000,2531.000000
unique,NaN,2529,NaN,NaN,NaN
top,NaN,CN=C1CN=C(C2=C(N1)C=CC(=C2)Cl)C3=CC=CN3,NaN,NaN,NaN
freq,NaN,2,NaN,NaN,NaN
mean,1.728745e+08,NaN,0.297906,0.297906,15.272651
std,1.258060e+08,NaN,0.457428,0.457428,11.989177
min,8.437060e+05,NaN,0.000000,0.000000,0.900000
25%,8.965011e+07,NaN,0.000000,0.000000,3.500000
50%,1.610040e+08,NaN,0.000000,0.000000,11.030000
75%,1.741614e+08,NaN,1.000000,1.000000,31.000000


### Perform data curation with `auroris.curation` module
The curation process includes:
- assign unique identifier to molecules
- detect the stereochemistry information of molecules.
- inspect the potential outliers of bioactivity values
- merge rows of replicated molecules
- detect isomers which show the activity shifts

Check out the curation module in [Auroris](https://github.com/polaris-hub/auroris). 

### Run preliminary curation for data inspection

In [12]:
# Define data column names
data_cols = [
    "PUBCHEM_ACTIVITY_OUTCOME",
    "PHENOTYPE",
    "HALF-LIFE",
]
mol_col = "SMILES"

In [13]:
# import key curation components from auroris
from auroris.curation import Curator
from auroris.curation.actions import MoleculeCuration, OutlierDetection, Deduplication, StereoIsomerACDetection, ContinuousDistributionVisualization

# Define the curation workflow
curator = Curator(
    data_path=source_data_path, 
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols = data_cols),
        ContinuousDistributionVisualization(y_cols=data_cols),
        OutlierDetection(method="zscore", columns=data_cols, threshold = 3, use_modified_zscore=True),
        StereoIsomerACDetection(y_cols=data_cols, threshold = 3)
    ],
    parallelized_kwargs = { "n_jobs": -1 }
)

curator.to_json(f"{dirname}/inspection_config.json")

In [14]:
# Run the curation step defined as above
data_inspection, report = curator(data)

2024-06-03 20:02:17.981 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation
2024-06-03 20:02:36.764 | INFO     | auroris.curation._curator:transform:106 - Performing step: distribution
2024-06-03 20:02:37.173 | INFO     | auroris.curation._curator:transform:106 - Performing step: outlier_detection
2024-06-03 20:02:37.418 | INFO     | auroris.curation._curator:transform:106 - Performing step: ac_stereoisomer


In [15]:
#  get the curation logger
from auroris.report.broadcaster import LoggerBroadcaster

broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

===== Curation Report =====
Time: 2024-06-03 20:02:17
Version: dev
===== mol_curation =====
[LOG]: New column added: MOL_smiles
[LOG]: New column added: MOL_molhash_id
[LOG]: New column added: MOL_molhash_id_no_stereo
[LOG]: New column added: MOL_num_stereoisomers
[LOG]: New column added: MOL_num_undefined_stereoisomers
[LOG]: New column added: MOL_num_defined_stereo_center
[LOG]: New column added: MOL_num_undefined_stereo_center
[LOG]: New column added: MOL_num_stereo_center
[LOG]: New column added: MOL_undefined_E_D
[LOG]: New column added: MOL_undefined_E/Z
[LOG]: Default `ecfp` fingerprint is used to visualize the chemical space.
[LOG]: Molecules with undefined stereocenter detected: 443.
[IMG]: Dimensions 2400 x 1200
[IMG]: Dimensions 1200 x 2400
===== distribution =====
[IMG]: Dimensions 1200 x 600
[IMG]: Dimensions 1200 x 600
[IMG]: Dimensions 1200 x 600
===== outlier_detection =====
[LOG]: New column added: OUTLIER_PUBCHEM_ACTIVITY_OUTCOME
[LOG]: Found 0 potential outliers with

In [16]:
# Generate an HTML report with embedded visualizations showcasing the data analysis.
from utils.auroris_utils import HTMLBroadcaster

# export report to local directory
broadcaster = HTMLBroadcaster(report, f"{dirname}/inspection_report")
report_path = broadcaster.broadcast()

In [18]:
# check the curated data
data_inspection.describe(include='all')

,PUBCHEM_SID,SMILES,PUBCHEM_ACTIVITY_OUTCOME,PHENOTYPE,HALF-LIFE,MOL_smiles,MOL_molhash_id,MOL_molhash_id_no_stereo,MOL_num_stereoisomers,MOL_num_undefined_stereoisomers,...,MOL_num_undefined_stereo_center,MOL_num_stereo_center,MOL_undefined_E_D,MOL_undefined_E/Z,OUTLIER_PUBCHEM_ACTIVITY_OUTCOME,OUTLIER_PHENOTYPE,OUTLIER_HALF-LIFE,AC_PUBCHEM_ACTIVITY_OUTCOME,AC_PHENOTYPE,AC_HALF-LIFE
count,2.531000e+03,2531,2531.000000,2531.000000,2531.000000,2531,2531,2531,2531.000000,2531.000000,...,2531.000000,2531.000000,2531,2531.0,2531,2531,2531,2531,2531,2531
unique,NaN,2529,NaN,NaN,NaN,2529,2529,2529,NaN,NaN,...,NaN,NaN,2,1.0,1,1,1,2,2,1
top,NaN,CN=C1CN=C(C2=C(N1)C=CC(=C2)Cl)C3=CC=CN3,NaN,NaN,NaN,CN=C1CN=C(c2ccc[nH]2)c2cc(Cl)ccc2N1,bb4504061cfb356324aa64c00169e1899b0622dc,ffd0d15b9fc35ea9be4baaa745606fecb913a336,NaN,NaN,...,NaN,NaN,False,0.0,False,False,False,False,False,False
freq,NaN,2,NaN,NaN,NaN,2,2,2,NaN,NaN,...,NaN,NaN,2120,2531.0,2531,2531,2531,2529,2529,2531
mean,1.728745e+08,NaN,0.297906,0.297906,15.272651,NaN,NaN,NaN,242.725010,4.590281,...,0.211774,0.434216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.258060e+08,NaN,0.457428,0.457428,11.989177,NaN,NaN,NaN,6512.604689,162.827079,...,0.583871,1.240903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,8.437060e+05,NaN,0.000000,0.000000,0.900000,NaN,NaN,NaN,1.000000,1.000000,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,8.965011e+07,NaN,0.000000,0.000000,3.500000,NaN,NaN,NaN,1.000000,1.000000,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.610040e+08,NaN,0.000000,0.000000,11.030000,NaN,NaN,NaN,1.000000,1.000000,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.741614e+08,NaN,1.000000,1.000000,31.000000,NaN,NaN,NaN,2.000000,1.000000,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### The readout `HALF-LIFE` is capped by value 31 which limits the usefulness of the normality plot.

![halflife](inspection_report/images/7-Outlier_detection_HALF-LIFE.png)

### Check activity shift between stereoisomers

In [19]:
data_inspection.loc[[213, 880]]

,PUBCHEM_SID,SMILES,PUBCHEM_ACTIVITY_OUTCOME,PHENOTYPE,HALF-LIFE,MOL_smiles,MOL_molhash_id,MOL_molhash_id_no_stereo,MOL_num_stereoisomers,MOL_num_undefined_stereoisomers,...,MOL_num_undefined_stereo_center,MOL_num_stereo_center,MOL_undefined_E_D,MOL_undefined_E/Z,OUTLIER_PUBCHEM_ACTIVITY_OUTCOME,OUTLIER_PHENOTYPE,OUTLIER_HALF-LIFE,AC_PUBCHEM_ACTIVITY_OUTCOME,AC_PHENOTYPE,AC_HALF-LIFE
213,109967254.0,CN=C1CN=C(C2=C(N1)C=CC(=C2)Cl)C3=CC=CN3,1.0,1.0,31.00,CN=C1CN=C(c2ccc[nH]2)c2cc(Cl)ccc2N1,bb4504061cfb356324aa64c00169e1899b0622dc,ffd0d15b9fc35ea9be4baaa745606fecb913a336,2,2,...,0,0,False,0,False,False,False,True,True,False
880,109967257.0,CN=C1CN=C(C2=C(N1)C=CC(=C2)Cl)C3=CC=CN3,0.0,0.0,22.99,CN=C1CN=C(c2ccc[nH]2)c2cc(Cl)ccc2N1,bb4504061cfb356324aa64c00169e1899b0622dc,ffd0d15b9fc35ea9be4baaa745606fecb913a336,2,2,...,0,0,False,0,False,False,False,True,True,False


Two possible stereochemistry based activity shifts were detected in the dataset.
Let's check the data in the dataset.

As we suspected, two samples with the same smiles but different SIDs are reported as being active or inactive. Looking at the molecules, they seem identical. We can't know which call is correct, so we should **remove** both. We'll do that below, after final curation.

SID 109967254 is [ML223](https://pubchem.ncbi.nlm.nih.gov/substance/109967254) (left). SID 109967257 is [NCGC00188362-03](https://pubchem.ncbi.nlm.nih.gov/substance/109967257) (right) (we've flipped the second molecule across the x axis to get it in the same orientation as the one on the left).

![image-2.png](inspection_report/images/8-Activity_shifts_among_stereoisomers__PUBCHEM_ACTIVITY_OUTCOME.png)

### Removing molecules as needed and re-run curation

In [20]:
# Make a fresh copy of the dataframe without the case where the curator has averaged scores.
data_to_curate = data.query(
    "SMILES != 'CN=C1CN=C(C2=C(N1)C=CC(=C2)Cl)C3=CC=CN3'"
).reset_index(drop=True)

## Rerun data curation and export curated data for downstream tasks

In [21]:
# Define the final curation workflow
curator = Curator(
    source_data=source_data_path, 
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols = data_cols),
        ContinuousDistributionVisualization(y_cols=data_cols),
        Deduplication(deduplicate_on=mol_col, y_cols=data_cols), # remove the replicated molecules
        OutlierDetection(method="zscore", columns=data_cols, threshold = 3, use_modified_zscore=True),
        StereoIsomerACDetection(y_cols=data_cols, threshold = 3)
    ],
    parallelized_kwargs = { "n_jobs": -1 }
)

In [22]:
# The final curation configuration is exported for reproducibility
path = f"{gcp_root}/data/curation/curation_config.json"
curator.to_json(path)

In [23]:
# Run the curation step defined as above
data_curated, report = curator(data_to_curate)

2024-06-03 20:02:40.643 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation
2024-06-03 20:02:49.639 | INFO     | auroris.curation._curator:transform:106 - Performing step: distribution
2024-06-03 20:02:50.100 | INFO     | auroris.curation._curator:transform:106 - Performing step: deduplicate
2024-06-03 20:02:51.597 | INFO     | auroris.curation._curator:transform:106 - Performing step: outlier_detection
2024-06-03 20:02:51.742 | INFO     | auroris.curation._curator:transform:106 - Performing step: ac_stereoisomer


In [24]:
broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

===== Curation Report =====
Time: 2024-06-03 20:02:40
Version: dev
===== mol_curation =====
[LOG]: New column added: MOL_smiles
[LOG]: New column added: MOL_molhash_id
[LOG]: New column added: MOL_molhash_id_no_stereo
[LOG]: New column added: MOL_num_stereoisomers
[LOG]: New column added: MOL_num_undefined_stereoisomers
[LOG]: New column added: MOL_num_defined_stereo_center
[LOG]: New column added: MOL_num_undefined_stereo_center
[LOG]: New column added: MOL_num_stereo_center
[LOG]: New column added: MOL_undefined_E_D
[LOG]: New column added: MOL_undefined_E/Z
[LOG]: Default `ecfp` fingerprint is used to visualize the chemical space.
[LOG]: Molecules with undefined stereocenter detected: 443.
[IMG]: Dimensions 2400 x 1200
[IMG]: Dimensions 1200 x 2400
===== distribution =====
[IMG]: Dimensions 1200 x 600
[IMG]: Dimensions 1200 x 600
[IMG]: Dimensions 1200 x 600
===== deduplicate =====
[LOG]: Deduplication merged and removed 1 duplicated molecules from dataset
===== outlier_detection ==

In [25]:
# Export report to polaris public directory on GCP
# The report is ready to reviewed in the HTML file.
broadcaster = HTMLBroadcaster(report, f"{gcp_root}/data/curation/report", embed_images=True)
broadcaster.broadcast()

'gs://polaris-public/polaris-recipes/org-polaris/ncats_adme/RLM/data/curation/report/index.html'

## Export the final curated data

In [26]:
fout = f"{gcp_root}/data/curation/{data_name}_curated.csv"
data_curated.reset_index(drop=True).to_csv(fout, index=False)

In [1]:
fout


NameError: name 'fout' is not defined

In [46]:
# Make a temporary directory to save the dataset
temp_dir = tempfile.TemporaryDirectory().name

save_dir = dm.fs.join(temp_dir, "dataset")

path = dataset.to_json(save_dir)

# Look at the save destination
fs = dm.fs.get_mapper(save_dir).fs
fs.ls(save_dir)

/Users/lu.zhu/miniconda3/envs/polaris_datasets/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


['/var/folders/_7/ffxc1f251dbb5msn977xl4sm0000gr/T/tmp8_vg7pfj/dataset/table.parquet',
 '/var/folders/_7/ffxc1f251dbb5msn977xl4sm0000gr/T/tmp8_vg7pfj/dataset/dataset.json']

In [47]:
folder = "04_ADME_NCATS"
data_curated.to_parquet(
    f"gs://polaris-private/curated_datasets/{folder}/{dataset.name}_curated.parquet"
)  # Save just in case. Requires a data folder.

save_dir = f"gs://polaris-private/Datasets/{folder}/{dataset.name}"
dataset.to_json(save_dir)

save_dir = f"gs://polaris-public/Datasets/{folder}/{dataset.name}"
dataset.to_json(save_dir)

# dataset.upload_to_hub()

'gs://polaris-public/Datasets/04_ADME_NCATS/ADME_NCATS_RLM_Stability/dataset.json'

In [48]:
d = Dataset.from_json('gs://polaris-public/Datasets/04_ADME_NCATS/ADME_NCATS_RLM_Stability/dataset.json')
d.license =  {"id": "CC-BY-4.0", "reference": "https://creativecommons.org/licenses/by/4.0/"}
d.upload_to_hub()

/Users/lu.zhu/miniconda3/envs/polaris_datasets/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


SSLCertVerificationError: ('We could not verify the SSL certificate. Please ensure the installed version (2024.02.02) of the `certifi` package is the latest. If you require the usage of a custom CA bundle, you can set the POLARIS_CA_BUNDLE environment variable to the path of your CA bundle. For debugging, you can temporarily disable SSL verification by setting the POLARIS_CA_BUNDLE environment variable to `false`.',)